# Eel imports

Now let's take a look at a cut of data on eel product imports. The data come from [a foreign trade database maintained by NOAA](https://www.st.nmfs.noaa.gov/commercial-fisheries/foreign-trade/).

The CSV file lives here: `../data/eels.csv`.

We'll start by importing pandas and creating a data frame.

In [ ]:
# import pandas


In [ ]:
# read csv into a data frame


In [ ]:
# check it out with head()


### Check out the values

Now let's poke through the values in each column to see what we're working with using a combination of `unique()`, `min()` and `max()`. Questions we're trying to answer here: What years and months are represented? What countries are in the data? Do the numeric data make sense? Are there any obvious errors or typos to handle? Are there any holes in our data (use `info()`)?

In [ ]:
# info()


In [ ]:
# unique year


In [ ]:
# unique month


In [ ]:
# unique countries


In [ ]:
# unique products => n.b.,
# have to use bracket notation bc "product" is a pandas function


**Question:** What does "ATC" stand for? _Always ask, never assume._

![atc](../img/eel-q.png "atc")

In [ ]:
# max/min for kilos


In [ ]:
# max/min for dollars


### Time-series data: Check for completeness

Each row in our data is one month's worth of shipments of a particular eel product from a particular country to the U.S. Which means we might want to do some time-based comparisons, which means we need to check that we're dealing with complete years.

So first let's think about what we want to see: For each year that's present in our data, we want a unique list of months for those records. If we were in Excel, we might pivot to group the data by `year` and then throw `month` in the "columns" section to see what months are present for each year.

We could also do a pivot table in pandas (stay tuned!), but I 
Here, we're going to do something similar:
- Select just the columns we're interested in
- Use the pandas `groupby()` method to group the records by year ([see this notebook for reference](../reference/Grouping%20data%20in%20pandas.ipynb))
- For each set of grouped data, use the pandas `unique()` method on the month column to see what months are present

When we call `groupby()` on a data frame, it returns a collection of items; each item in that collection is a Python tuple with two elements: the _grouping_ value (year, in this case) and a data frame of records that belong to that group (all records where year == that year).

For our purposes, that means we can use a _for loop_ to iterate over the results and check each year.

👉For more details on _for loops_, [see this notebook](../reference/Python%20data%20types%20and%20basic%20syntax.ipynb#for-loops).

In [ ]:
# select year and month columns

# for loop to iterate over yearmonth grouped by 'year' column
for yeargroup in yearmonth.groupby('year'):
    # print the first [0] item in the grouping
    
    # in the second [1] item in the grouping (the grouped dataframe)
    # print a unique() list of months


So now we know that we have incomplete data for 2017 -- _news we can use_ as we start our analysis.

### Come up with a list of questions

- In this data, what country ships the most eel products of any type to the U.S.?
- Same question but broken out by year.
- For each country, what was the percent change in eel shipments of all types from 2010-2016?
- What type of product is most popular?

### Q: Who ships the most eels to the U.S. (in kilos)?

We'll use our good friends `groupby()`, `sum()` and `sort_values()`to find out.

In [ ]:
# select country and kilos columns
# groupby country column
# run sum()
# sort values by kilos descending


### Q: Who ships the most? (Broken out by year)

Now we want to create a table where the rows are countries, the columns are years, and the values are sums for that country, that year.

If we were doing this in an Excel pivot table, we'd just add "year" to the columns section. To do this in pandas, we're ... also going to use a pivot table. (Yes! Pandas has a [pivot table function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html).)

We are going to hand the `pivot_table()` function five arguments:
- The data we're pivoting (`df`)
- The type of aggregation to apply to the values -- default is `mean` and we do not want that (`aggfunc='sum'`)
- The name of the column whose values we're doing math on (`values='kilos'`)
- The name of the column we're grouping on (`index='country'`)
- The name of the column whose values will become the columns of our table (`columns='year'`)

Then we'll use `sort_values()` to sort the pivot table that results by our most recent year of data.

In [ ]:
# run a pivot table on df
# aggfunc is sum
# values is kilos column
# index is country column
# columns is year column
# save pivot table as new variable

# sort that pivot table by 2017 column descending


### Q: What was the percent change in shipments for each country from 2010-2016?

For this question, we'll re-use the pivot table we just made and add a calculated column. First, though, we need to filter the table to include only records where the `2010` and `2016` values are not null, using the [`notnull()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.notnull.html). (Looks like just filtering for "2010 is not null" does the trick.)

**If you didn't know about `.notnull()` already, how would you Google to find the answer?**

In [ ]:
# filter to get pivoted sums where the 2010 column is notnull()


Now we can add a column -- `10to16pctchange`. The syntax, and the math -- new value minus old value divided by old value -- are relatively straightforward: 

`dataframe['new_column'] = (dataframe['new_value'] - dataframe['old_value']) / dataframe['old_value'] * 100`

You might get a warning about slices vs. copies. You can ignore that for now.

In [ ]:
# create a new column, 10to16pctchange
# new value is 2016
# old value is 2010


In [ ]:
# check it out, sorted by our new column


### Q: What type of product is most popular (in kilos)?

We'll use `groupby()`, `sum()` and `sort_values()` again.

In [ ]:
# select product and kilos columns
# groupby product column
# run sum()
# sort values by kilos descending


### How many kilos of eels were imported from Central and South America?

We'll need to filter the data to get just shipments from Central and South American countries; for this, we'll use a filtering method called [`isin()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.isin.html).

👉 For more details on filtering in pandas, [check out this notebook](../reference/Filtering%20columns%20and%20rows%20in%20pandas.ipynb).

First, let's filter our main data frame to get just Central and South American countries. Let's take a look at what countries are present in our data, again:

In [ ]:
# get unique list of country column


So it looks like we want `PANAMA`, `CHILE` and `COSTA RICA`. The `isin()` method expects a list, so that's what we'll hand it.

👉 For more information on lists, [check out this notebook](../reference/Python%20data%20types%20and%20basic%20syntax.ipynb#Lists).

In [ ]:
# make a list of our countries of interest

# use isin() to filter for those countries


In [ ]:
# check out the results with head()


Now we can run the `sum()` of the `kilos` column to get our answer:

In [ ]:
# sum of kilos column on our filtered data


# 📚 GROUP HOMEWORK 📚

In your group, answer these questions:
- What's the average and median value, in dollars, of a shipment?
- Fill in the blank: "The U.S. imported ${} worth of eels from Europe from 2010 to mid-2017." (i.e., across the whole data set we're looking at)